In [122]:


fl = read.csv("featurelabels.csv")
head(fl)
print(fl[fl$Variable=="F09787",]$LabelDefinition)

,Variable,LabelDefinition
,<chr>,<chr>
1,YR,Calendar Year
2,F00002,Federal Income Processing System (FIPS) code - unique 5-digit county identifier
3,F12424,State name abbreviation
4,F00010,County name
5,F04437,County name with state abberviation
6,F13874,Total area


[1] "Healthcare Professional Shortage Area (Primary Care) "


In [187]:

ORD_DEATHS = read.csv("ORD_DEATHS_results_all_years.csv",header=T, fileEncoding = 'UTF-8-BOM')
head(ORD_DEATHS)


,X,State.Abbv,F00020,F09787,F14642,F13214,F13220,F11984,F04538,F04542,...,SPEC_PC,PCT_25T44,PCT_45T64,PCT_65PLUS,EXP_EARLY,ORD_CDR,CANCER_CDR,NP_RX,PDMP_REQ_CHECK,State
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,0,AK,0.00000000,0,0.0000000,0,0.007180287,0,0,0.00000000,...,0,0.000000000,0,0.000000000,0,0.1195591353,0,0,0,Alaska
2,1,AL,-0.05603074,0,0.3901242,0,0.000000000,0,0,0.01758385,...,0,0.000000000,0,0.000000000,0,0.1895658697,0,0,0,Alabama
3,2,AR,-0.03795153,0,0.0000000,0,0.000000000,0,0,0.00000000,...,0,0.001483639,0,-0.007003101,0,0.1107100063,0,0,0,Arkansas
4,3,AZ,0.00000000,0,0.0000000,0,0.000000000,0,0,0.00000000,...,0,0.000000000,0,0.000000000,0,0.0274693759,0,0,0,Arizona
5,4,CA,0.00000000,0,0.0000000,0,0.000000000,0,0,0.00000000,...,0,0.000000000,0,0.000000000,0,0.0242500825,0,0,0,California
6,5,CO,0.00000000,0,0.0000000,0,0.000000000,0,0,0.00000000,...,0,0.000000000,0,0.000000000,0,0.0003351251,0,0,0,Colorado


In [197]:
#remove columns that are all 0
removal = c()
for(i in (1:length(ORD_DEATHS))){
    if(all(ORD_DEATHS[i]==0)){
        removal = append(removal,i)
    }
}
if(length(removal) >0){
    ORD_DEATHS = subset(ORD_DEATHS,select = -removal)
}
ORD_DEATHS_output = ORD_DEATHS

#instantiate "Rank" columns
max_features = length(ORD_DEATHS[3,])-1
temp = ORD_DEATHS[,2:max_features]
for (each in names(temp)[2:145]){
    # print(each)
    if(each %in% fl$Variable){
        colname = paste("RANK",fl[fl$Variable==each,]$LabelDefinition)
        names(temp)[names(temp)==each] <- colname
        temp[colname] = "NA"
    }
}

#relabel names according to what the feature labels actually mean, denote as "COEFF", to distinguish from "RANK" later.
for (each in names(ORD_DEATHS)){
    if(each %in% fl$Variable){
        names(ORD_DEATHS_output)[names(ORD_DEATHS_output)==each] <- paste("COEFF",fl[fl$Variable==each,]$LabelDefinition)
    }
}
# print(names(ORD_DEATHS_output))
# print(names(temp))
ORD_DEATHS_output = merge(ORD_DEATHS_output,temp,by="State.Abbv")
# print(names(ORD_DEATHS_output))
# print(ORD_DEATHS[1,][143])

#determine rank of coefficient fields with positive value
for(each in (1:length(ORD_DEATHS$X))){
    rowvalues = ORD_DEATHS[each,][3:max_features][sapply(ORD_DEATHS[each,][3:max_features],function(x) x > 0)]
    for(i in (1:length(rowvalues))){
        #determine rank, apply to corresponding RANK column
        thisvalue = rowvalues[i]
        thiscolumnname = names(rowvalues)[i]
        thisrank = length(rowvalues[1:length(rowvalues)][sapply(rowvalues[1:length(rowvalues)],function(x) x > thisvalue)])+1
        ORD_DEATHS_output[each,paste("RANK",fl[fl$Variable==thiscolumnname,]$LabelDefinition)] = thisrank
    }
}





head(ORD_DEATHS_output)

Warning message in merge.data.frame(ORD_DEATHS_output, temp, by = "State.Abbv"):
"column names 'COEFF % Black ', 'COEFF % Hispanic ' are duplicated in the result"


,State.Abbv,X,COEFF USDA Rural-Urban Continuum Code,COEFF Healthcare Professional Shortage Area (Primary Care),COEFF # of nurse practitioners with National Provider Identifiers (NPI),COEFF # of home health agencies,COEFF # of hospices,COEFF % Black,COEFF % Hispanic,COEFF Veteran population estimate,...,RANK Dentists aged 65+ per 100000 residents,RANK Medical doctors per 100000 residents,RANK Specialists per 100000 residents,RANK % aged 25 to 44,RANK % aged 45 to 64,RANK % aged 65+,RANK Crude opiod-related death rate - imputed,RANK Crude cancer-related death rate - imputed,RANK Nurse practitioner prescribing authority,RANK Presription drug monitoring programs (PDMP)
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,AK,0,0.00000000,0,0.0000000,0,0.007180287,0,0.00000000,0.0000000,...,NA,NA,NA,NA,NA,NA,2,NA,NA,NA
2,AL,1,-0.05603074,0,0.3901242,0,0.000000000,0,0.01758385,0.0000000,...,NA,NA,NA,NA,NA,NA,3,NA,NA,NA
3,AR,2,-0.03795153,0,0.0000000,0,0.000000000,0,0.00000000,0.0000000,...,NA,NA,NA,7,NA,NA,3,NA,NA,NA
4,AZ,3,0.00000000,0,0.0000000,0,0.000000000,0,0.00000000,0.0000000,...,NA,NA,NA,NA,NA,NA,6,NA,NA,NA
5,CA,4,0.00000000,0,0.0000000,0,0.000000000,0,0.00000000,0.7015755,...,NA,NA,NA,NA,NA,NA,5,NA,NA,NA
6,CO,5,0.00000000,0,0.0000000,0,0.000000000,0,0.00000000,0.0000000,...,NA,NA,NA,NA,NA,NA,9,NA,NA,NA


In [198]:

PCPV = read.csv("PCPV_results_all_years.csv",header=T, fileEncoding = 'UTF-8-BOM')
head(PCPV)


,X,State.Abbv,F00020,F09787,F14642,F13214,F13220,F11984,F04538,F04542,...,SPEC_PC,PCT_25T44,PCT_45T64,PCT_65PLUS,EXP_EARLY,ORD_CDR,CANCER_CDR,NP_RX,PDMP_REQ_CHECK,State
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,0,AK,0,0,0.0000000,0.00000000,0.02253174,0,0,0.0000000,...,0,0,0.000000000,0,0.00000000,0.0000000,0.00000000,0,0,Alaska
2,1,AL,0,0,0.0000000,0.03832035,0.02683585,0,0,0.0000000,...,0,0,0.000000000,0,0.00000000,0.5122997,0.00000000,0,0,Alabama
3,2,AR,0,0,0.0000000,0.00000000,0.08327911,0,0,0.0000000,...,0,0,-0.055734457,0,0.00000000,0.5366140,-0.03502457,0,0,Arkansas
4,3,AZ,0,0,0.0000000,0.00000000,0.00000000,0,0,0.0000000,...,0,0,0.000000000,0,0.00000000,0.1571522,0.22449476,0,0,Arizona
5,4,CA,0,0,0.0000000,0.00000000,0.00000000,0,0,-0.5031370,...,0,0,-0.008476857,0,0.06374002,0.1339255,0.00000000,0,0,California
6,5,CO,0,0,-0.1142714,0.00000000,0.00000000,0,0,0.1770186,...,0,0,0.000000000,0,0.00000000,0.1601276,0.10386516,0,0,Colorado


In [199]:
#remove columns that are all 0
removal = c()
for(i in (1:length(PCPV))){
    if(all(PCPV[i]==0)){
        removal = append(removal,i)
    }
}
if(length(removal) >0){
    PCPV = subset(PCPV,select = -removal)
}
PCPV_output = PCPV

#instantiate "Rank" columns
max_features = length(PCPV[3,])-1
temp = PCPV[,2:max_features]
for (each in names(temp)[2:145]){
    if(each %in% fl$Variable){
        colname = paste("RANK",fl[fl$Variable==each,]$LabelDefinition)
        names(temp)[names(temp)==each] <- colname
        temp[colname] = "NA"
    }
}
#relabel names according to what the feature labels actually mean, denote as "COEFF", to distinguish from "RANK" later.
for (each in names(PCPV)){
    if(each %in% fl$Variable){
        names(PCPV_output)[names(PCPV_output)==each] <- paste("COEFF",fl[fl$Variable==each,]$LabelDefinition)
    }
}
PCPV_output = merge(PCPV_output,temp,by="State.Abbv")

#determine rank of coefficient fields with positive value
for(each in (1:length(PCPV$X))){
    rowvalues = PCPV[each,][3:max_features][sapply(PCPV[each,][3:max_features],function(x) x > 0)]
    for(i in (1:length(rowvalues))){
        #determine rank, apply to corresponding RANK column
        thisvalue = rowvalues[i]
        thiscolumnname = names(rowvalues)[i]
        thisrank = length(rowvalues[1:length(rowvalues)][sapply(rowvalues[1:length(rowvalues)],function(x) x > thisvalue)])+1
        PCPV_output[each,paste("RANK",fl[fl$Variable==thiscolumnname,]$LabelDefinition)] = thisrank
    }
}


head(PCPV_output)

Warning message in merge.data.frame(PCPV_output, temp, by = "State.Abbv"):
"column names 'COEFF % Black ', 'COEFF % Hispanic ' are duplicated in the result"


,State.Abbv,X,COEFF USDA Rural-Urban Continuum Code,COEFF Healthcare Professional Shortage Area (Primary Care),COEFF # of nurse practitioners with National Provider Identifiers (NPI),COEFF # of home health agencies,COEFF # of hospices,COEFF % Black,COEFF % Hispanic,COEFF Veteran population estimate,...,RANK Dentists aged 55 to 64 per 100000 residents,RANK Dentists aged 65+ per 100000 residents,RANK % aged 25 to 44,RANK % aged 45 to 64,RANK % aged 65+,RANK Early state Medicaid expansion status,RANK Crude opiod-related death rate - imputed,RANK Crude cancer-related death rate - imputed,RANK Nurse practitioner prescribing authority,RANK Presription drug monitoring programs (PDMP)
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,AK,0,0,0,0.0000000,0.00000000,0.02253174,0,0.0000000,0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,AL,1,0,0,0.0000000,0.03832035,0.02683585,0,0.0000000,0,...,10,NA,NA,NA,NA,NA,2,NA,NA,NA
3,AR,2,0,0,0.0000000,0.00000000,0.08327911,0,0.0000000,0,...,NA,NA,NA,NA,NA,NA,2,NA,NA,NA
4,AZ,3,0,0,0.0000000,0.00000000,0.00000000,0,0.0000000,0,...,2,NA,NA,NA,NA,NA,4,3,NA,NA
5,CA,4,0,0,0.0000000,0.00000000,0.00000000,0,-0.5031370,0,...,NA,4,NA,NA,NA,5,3,NA,NA,NA
6,CO,5,0,0,-0.1142714,0.00000000,0.00000000,0,0.1770186,0,...,16,20,NA,NA,NA,NA,4,9,NA,NA


In [200]:
write.csv(ORD_DEATHS_output,file="ORD_DEATHS_with_rank2.csv")
write.csv(PCPV_output,file="PCPV_with_rank2.csv")